CA02: This is a eMail Spam Classifers that uses Naive Bayes supervised machine learning algorithm. 

In this assignment you will ...
1. Complete the code such a way that it works correctly with this given parts of the program.
2. Explain as clearly as possible what each part of the code is doing. Use "Markdown" texts and code commenting to explain the code

IMPORTANT NOTE:

The path of your data folders 'train-mails' and 'test-mails' must be './train-mails' and './test-mails'. This means you must have your .ipynb file and these folders in the SAME FOLDER in your laptop or Google Drive. The reason for doing this is, this way the peer reviewes and I would be able to run your code from our computers using this exact same relative path, irrespective of our folder hierarchy.

## Libraries to import

### Summary of each library imported and its function.

1. **nltk.tokenize**: This function is used to tokenize text into individual words.
2. **pos_tag**: Used for Parts of Speech tagging
3. **stopwords**: provides a list of english words that mainly serve a grammatical function rather than provide meaning on their own.
4. **WordNetLemmatizer**: performs word lemmatization which maps all the different forms of a word to its base form.
5. **LabelEncoder**: Encodes the text into numerical variables that are digestible by the machine learning algorithms.
6. **defaultdict**: useful in counting and aggregating data by using methods such as place holders for null key values.
7. **wordnet**: used for clarifying meaning in words and finding appropriate synonyms.
8. **TfidfVectorizer**: used for feature engineering, particularly on converting words into a numerical score using Term Frequency-Inverse Document Frequency (TF-IDF), to calculate the score of a word based on its frequency and rarity in appearance. The more frequent it is, the higher the score. The more rare it is, the higher the score. Words with high TF and IDF are ranked highest in a document.
9. **model_selection**, naive_bayes, svm: importaing the machine learning algorithms to be used in training and testing the data. Naive Bayes is a probability based algorithm founded on Bayes principles that assumes each word is an independent event and assigns a probability to each word of its likelhood of belonging to a certain class. Support Vector Machines (SVM) find the most ideal between text categories that maximizes the distance from the boundary to enhance accuracy.
10. **accuracy_score**: Determines the accuracy of the applied machine learning algorithms 

**NB If you are using Jupyter Notebook, ensure you install the libraries in the Anaconda environment, or by using the !pip function. See the code below for reference.**


In [1]:
import os
import numpy as np
from collections import Counter

# Import all other necessary libraries. Your code below ...

# Below are the additional libraries required for NLP text classification. 
# Additionally, requests, io, and nltk need to be imported.
# Finally, we download the various nltk methods. Refer to code below.
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

!pip install requests

import requests
import io
import nltk
nltk.download('punkt') # necessary on step (c), tokenization
nltk.download('wordnet') # necessary on step (d), stop words
nltk.download('omw-1.4') # necessary on step (d), stop words
nltk.download('averaged_perceptron_tagger') # necessary on step (d), stop words
nltk.download('stopwords') # necessary on step (d), stop words


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zionn\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\zionn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\zionn\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\zionn\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zionn\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Refer to comments in the cell below.

In [2]:
# In summary, a function is being created to load the numerous text files then extract the content from each one
# before cleaning the text using methods such as tokenization. Next, the words are put into a Dictionary, which
# counts the frequency of each word. Finally, the most common 3000 words are kept in the dictionary after a process 
# of elimination.

def make_Dictionary(root_dir):
  all_words = []
  emails = [os.path.join(root_dir,f) for f in os.listdir(root_dir)] #Collect txt files in root directory and store them in emails. 
  for mail in emails:
    with open(mail) as m:
      for line in m:
        words = line.split()
        all_words += words #The FOR loop runs through each line, tokenizes the words, and stores them into a list called all_words.
  dictionary = Counter(all_words) # Counter transforms "all_words" into a dictionary with the words and their frequency of appearance.
  list_to_remove = list(dictionary) # A list is created to be used for removal of unecessary words or characters.

  for item in list_to_remove:
    if item.isalpha() == False: # FOR loop checks whether words in "list_to_remove" are alpha characters.
      del dictionary[item] # If not an alpha character, it is deleted. 
    elif len(item) == 1:  # Identifies words that are one character long. 
      del dictionary[item] # If the word is one character long, it is removed. An example is 'a'. 
  dictionary = dictionary.most_common(3000) # A pre-defined list in the packages is used to retain the 3000 most common words.
  return dictionary
            

## Summary of function `def extract features`

1. The first line within the function extracts the text files in the folders and stores them in a variable called '_Files_'.
2. Next, it creates a variable called '_feature_matrix_', which is a matrix with the number of zeros equal to the length of text files loaded in the previous step in '_files_'.
3. Another variable is created called '_train_labels_' that has the same number of zeros as the number of text files.
4. A FOR loop is created that runs through each file, and another FOR loop within it goes through the 3rd line of each file.
5. The 3rd line is then split into words, which are then cross referenced in the '_dictionary_' and if there is a match the word is assigned to '_wordID_'. The '_wordID_' variable will be used in feature engineering. 
6. A count of the words is done in the line of code '_features_matrix[docID,wordID] = words.count(word)_' to determine the frequency of the words appearing the both the email and dictionary.
7. The lines of code from '_train_labels_' to '_lastToken_' extract the file names and store them in lastToken, which will be used to determine if the file is spam or not. 
8. AN IF statement is then used to identify any text starting with '_spmsg_' and labels it as 1. The docID keeps track of the email being processed. 




In [3]:
def extract_features(mail_dir):
  files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
  features_matrix = np.zeros((len(files),3000))
  train_labels = np.zeros(len(files))
  count = 1;
  docID = 0;
  for fil in files:
    with open(fil) as fi:
      for i, line in enumerate(fi):
        if i ==2:
          words = line.split()
          for word in words:
            wordID = 0
            for i, d in enumerate(dictionary):
              if d[0] == word:
                wordID = i
                features_matrix[docID,wordID] = words.count(word)
      train_labels[docID] = 0;
      filepathTokens = fil.split('/')
      lastToken = filepathTokens[len(filepathTokens)-1]
      if lastToken.startswith("spmsg"):
        train_labels[docID] = 1;
        count = count + 1
      docID = docID + 1
  return features_matrix, train_labels                

###  Train and Test Data

The lines of code below are used for creating storage paths for the folders containing the training and testing emails, which will be used in the Naives Bayes algorithm.

In [4]:
# Enter the "path" of your "train_mails" and "test-mails" FOLDERS in this cell ...
# for example: TRAIN_DIR = '../../train-mails'
#              TEST_DIR = '../../test-mails'

TRAIN_DIR = 'C:/Users/zionn/msba/CA02/Data/train-mails'
TEST_DIR = 'C:/Users/zionn/msba/CA02/Data/test-mails'

### Creating the Features Matrix

Next the most common 3000 words are called from the '_TRAIN_DIR_'  into dictionary. A statement indicating that the '_reading and processing emails from TRAIN and TEST folders_' is printed in case the algorithm takes a long time to run and anyone reading the code knows at what stage the process is.

Apply the extract_features function is applied to the train and test directories and returns a feature matrix adn labels. The feature matrix shows the frequency of the 3000 most common words in each email and the labels show the type of emails, i.e. 1 for spam emails, and 0 for non-spam emails. 

Finally, the variables '_test_features_matrix_' and '_test_labels_' are used later to evaluate the performance of the model in classifying the data correctly. 

In [5]:
dictionary = make_Dictionary(TRAIN_DIR)

print ("reading and processing emails from TRAIN and TEST folders")
features_matrix, labels = extract_features(TRAIN_DIR)
test_features_matrix, test_labels = extract_features(TEST_DIR)

reading and processing emails from TRAIN and TEST folders


In [6]:
# In this section enter your code to TRAIN the model using Naive Bayes algorithm, then PREDICT and then evaluate PERFORMANCE (Accuracy)
# Your code below ...
#
#
#
# Your output should look like below if your code is right

# Importing the Naives Algorithm and 
from sklearn.naive_bayes import GaussianNB

# Initialize the classifier
gnb = GaussianNB()

# Fit the classifier on the extracted features and corresponding labels
print ("Training Model using Gaussian Naive Bayes algorithm .....")
gnb.fit(features_matrix, labels)
print ("Training completed")



Training Model using Gaussian Naive Bayes algorithm .....
Training completed


In [7]:
# Use the trained classifier to predict the labels of the test data
print ("testing trained model to predict Test Data labels")
predicted_labels = gnb.predict(test_features_matrix)
print ("Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:")


testing trained model to predict Test Data labels
Completed classification of the Test Data .... now printing Accuracy Score by comparing the Predicted Labels with the Test Labels:


### Training and evaluation the model


After the model has been successfully trained in the above code, we evaluate its performance in the code below. This is done by importing '_accuracy_score_' from the sklearn.metrics library. This function will compare the predicted values with the actual values and determine an accuracy score.  

#### Results:

The accuracy returned is 1.0, which implies a 100% accuracy. This figure needs to be confirmed because it is rare to get model score of this level of accuracy. 

In [8]:
# Import functions to evaluate performance of the model. 

# Accuracy score measures the accuracy of correct predictions. 
from sklearn.metrics import accuracy_score
print("Accuracy:", accuracy_score(test_labels, predicted_labels))


Accuracy: 1.0


======================= END OF PROGRAM =========================